# Diophantine Constraint Exploration: Resource Allocation

**Problem**: Count nonnegative integer solutions to:
- $x_1 + x_2 + \cdots + x_n = u$
- $x_1 + 2x_2 + 3x_3 + \cdots + nx_n = v$

**Multiplicity**: $M(u,v) = \#\{\mathbf{x} \in \mathbb{Z}_{\ge 0}^n : \text{constraints satisfied}\}$

## Notebook 0 — Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sympy import symbols, Eq, solve, diophantine
from itertools import product
import json
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Create directories
Path('results').mkdir(exist_ok=True)
Path('figures').mkdir(exist_ok=True)

print("Setup complete.")

In [ ]:
# Experiment configuration
CONFIG = {
    'n': 3,  # Number of tasks/variables
    'u_max': 50,  # Maximum total demand
    'v_max': 150,  # Maximum weighted capacity (n * u_max)
    'u_range': range(0, 51),  # Range for u
    'v_range': range(0, 151),  # Range for v
    'cache_enabled': True,  # Cache results
    'cache_file': 'results/multiplicity_cache.json'
}

print(f"Configuration: n={CONFIG['n']}, u_max={CONFIG['u_max']}, v_max={CONFIG['v_max']}")

## Notebook 1 — Define the Constraint

In [ ]:
def define_constraint(n):
    """
    Define the Diophantine constraint symbolically.
    
    Returns:
        constraint_func: Function that checks if x satisfies constraints for given (u,v)
    """
    def constraint_func(x, u, v):
        """
        Check if vector x satisfies:
        - sum(x) = u
        - sum(i * x[i-1]) = v  (1-indexed to 0-indexed conversion)
        """
        if len(x) != n:
            return False
        
        # Check nonnegativity
        if any(xi < 0 for xi in x):
            return False
        
        # Constraint 1: sum = u
        if sum(x) != u:
            return False
        
        # Constraint 2: weighted sum = v
        weighted_sum = sum((i+1) * x[i] for i in range(n))
        if weighted_sum != v:
            return False
        
        return True
    
    return constraint_func

# Create constraint checker
check_constraint = define_constraint(CONFIG['n'])

print(f"Constraint defined for n={CONFIG['n']}")
print(f"Constraint 1: x_1 + x_2 + ... + x_n = u")
print(f"Constraint 2: x_1 + 2*x_2 + ... + n*x_n = v")

In [ ]:
# Test the constraint
test_x = [2, 3, 0]
test_u = 5
test_v = 8
result = check_constraint(test_x, test_u, test_v)
print(f"Test: x={test_x}, u={test_u}, v={test_v}")
print(f"Satisfies constraint: {result}")
print(f"Check: sum={sum(test_x)}, weighted_sum={sum((i+1)*test_x[i] for i in range(len(test_x)))}")

## Notebook 2 — Enumerator / Solver

In [ ]:
def enumerate_solutions_direct(n, u, v, max_iter=1000000):
    """
    Direct enumeration: iterate over all possible x vectors.
    Pruned by bounds: x_i <= u for all i.
    """
    solutions = []
    
    # Bounds: each x_i can be at most u
    bounds = [u + 1] * n
    
    count = 0
    for x in product(*[range(b) for b in bounds]):
        count += 1
        if count > max_iter:
            break
        
        x_list = list(x)
        if check_constraint(x_list, u, v):
            solutions.append(x_list)
    
    return solutions

In [ ]:
def enumerate_solutions_pruned(n, u, v):
    """
    Pruned enumeration using constraint structure.
    
    From x_1 + ... + x_n = u and x_1 + 2x_2 + ... + nx_n = v,
    we can eliminate variables to reduce search space.
    """
    solutions = []
    
    # For n=3: x_1 + x_2 + x_3 = u, x_1 + 2x_2 + 3x_3 = v
    # Subtracting: x_2 + 2x_3 = v - u
    # So x_2 = (v - u) - 2x_3, and x_1 = u - x_2 - x_3
    
    if n == 3:
        for x3 in range(u + 1):
            x2 = (v - u) - 2 * x3
            if x2 < 0:
                continue
            x1 = u - x2 - x3
            if x1 < 0:
                continue
            
            x = [x1, x2, x3]
            if check_constraint(x, u, v):
                solutions.append(x)
    else:
        # General case: use direct enumeration for small n
        return enumerate_solutions_direct(n, u, v)
    
    return solutions

In [ ]:
# Test enumeration
test_solutions = enumerate_solutions_pruned(CONFIG['n'], 10, 15)
print(f"Number of solutions for (u=10, v=15): {len(test_solutions)}")
print(f"Sample solutions: {test_solutions[:5]}")

## Notebook 3 — Multiplicity Definition & Computation

In [ ]:
def compute_multiplicity(n, u, v, use_cache=True, cache=None):
    """
    Compute M(u,v) = number of solutions.
    """
    # Check cache
    if use_cache and cache is not None:
        key = f"{n}_{u}_{v}"
        if key in cache:
            return cache[key]
    
    # Check feasibility bounds
    if v < u or v > n * u:
        result = 0
    else:
        solutions = enumerate_solutions_pruned(n, u, v)
        result = len(solutions)
    
    # Store in cache
    if use_cache and cache is not None:
        key = f"{n}_{u}_{v}"
        cache[key] = result
    
    return result

In [ ]:
# Load or create cache
cache = {}
if CONFIG['cache_enabled'] and Path(CONFIG['cache_file']).exists():
    with open(CONFIG['cache_file'], 'r') as f:
        cache = json.load(f)
    print(f"Loaded cache with {len(cache)} entries")
else:
    print("Starting with empty cache")

In [ ]:
# Compute multiplicity matrix
print("Computing multiplicity matrix...")
multiplicity_matrix = np.zeros((CONFIG['u_max'] + 1, CONFIG['v_max'] + 1), dtype=int)

for u in tqdm(CONFIG['u_range'], desc="Computing M(u,v)"):
    for v in CONFIG['v_range']:
        multiplicity_matrix[u, v] = compute_multiplicity(CONFIG['n'], u, v, cache=cache)

print("Computation complete.")

In [ ]:
# Save cache
if CONFIG['cache_enabled']:
    with open(CONFIG['cache_file'], 'w') as f:
        json.dump(cache, f)
    print(f"Cache saved with {len(cache)} entries")

In [ ]:
# Export sample solutions to CSV
sample_data = []
for u in range(0, min(21, CONFIG['u_max'] + 1)):
    for v in range(u, min(n * u + 1, CONFIG['v_max'] + 1)):
        count = multiplicity_matrix[u, v]
        if count > 0:
            sample_data.append({'u': u, 'v': v, 'multiplicity': count})

df = pd.DataFrame(sample_data)
df.to_csv('results/multiplicity_samples.csv', index=False)
print(f"Exported {len(df)} sample entries to CSV")
print(df.head(10))

## Notebook 4 — Heatmaps

In [ ]:
# Create heatmap
fig, ax = plt.subplots(figsize=(12, 10))

# Clip to feasible region for better visualization
u_display = min(50, CONFIG['u_max'])
v_display = min(150, CONFIG['v_max'])

heatmap_data = multiplicity_matrix[:u_display+1, :v_display+1]

# Use log scale for better visualization (add 1 to avoid log(0))
heatmap_log = np.log1p(heatmap_data)

im = ax.imshow(heatmap_log.T, aspect='auto', origin='lower', cmap='viridis', interpolation='nearest')
ax.set_xlabel('u (Total Demand)', fontsize=12)
ax.set_ylabel('v (Weighted Capacity)', fontsize=12)
ax.set_title(f'Multiplicity Heatmap M(u,v) for n={CONFIG["n"]} (log scale)', fontsize=14)

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('log(1 + M(u,v))', fontsize=10)

plt.tight_layout()
plt.savefig('figures/heatmap_log.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Linear scale heatmap (zoomed to interesting region)
fig, ax = plt.subplots(figsize=(12, 10))

im = ax.imshow(heatmap_data.T, aspect='auto', origin='lower', cmap='YlOrRd', interpolation='nearest')
ax.set_xlabel('u (Total Demand)', fontsize=12)
ax.set_ylabel('v (Weighted Capacity)', fontsize=12)
ax.set_title(f'Multiplicity Heatmap M(u,v) for n={CONFIG["n"]} (linear scale)', fontsize=14)

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('M(u,v)', fontsize=10)

plt.tight_layout()
plt.savefig('figures/heatmap_linear.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Slice plots: M(u,v) for fixed u
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

u_values = [10, 20, 30, 40]
for idx, u in enumerate(u_values):
    if u <= CONFIG['u_max']:
        v_range_plot = range(u, min(CONFIG['n'] * u + 1, CONFIG['v_max'] + 1))
        m_values = [multiplicity_matrix[u, v] for v in v_range_plot]
        
        axes[idx].plot(list(v_range_plot), m_values, marker='o', markersize=3)
        axes[idx].set_xlabel('v (Weighted Capacity)', fontsize=10)
        axes[idx].set_ylabel('M(u,v)', fontsize=10)
        axes[idx].set_title(f'Slice at u={u}', fontsize=11)
        axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('figures/slices_fixed_u.png', dpi=300, bbox_inches='tight')
plt.show()

## Notebook 5 — Geometry Diagnostics

In [ ]:
# For n=3, plot solution projections in 3D space
if CONFIG['n'] == 3:
    from mpl_toolkits.mplot3d import Axes3D
    
    # Collect solutions for a few (u,v) pairs
    fig = plt.figure(figsize=(15, 5))
    
    test_pairs = [(10, 15), (20, 30), (30, 50)]
    
    for idx, (u, v) in enumerate(test_pairs):
        ax = fig.add_subplot(1, 3, idx + 1, projection='3d')
        
        solutions = enumerate_solutions_pruned(3, u, v)
        
        if solutions:
            x1_vals = [s[0] for s in solutions]
            x2_vals = [s[1] for s in solutions]
            x3_vals = [s[2] for s in solutions]
            
            ax.scatter(x1_vals, x2_vals, x3_vals, s=50, alpha=0.6)
            ax.set_xlabel('x1', fontsize=9)
            ax.set_ylabel('x2', fontsize=9)
            ax.set_zlabel('x3', fontsize=9)
            ax.set_title(f'u={u}, v={v}\n{len(solutions)} solutions', fontsize=10)
    
    plt.tight_layout()
    plt.savefig('figures/geometry_3d.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# Compute invariants: parity analysis
print("Parity Analysis:")
print("=" * 50)

parity_counts = {'even_even': 0, 'even_odd': 0, 'odd_even': 0, 'odd_odd': 0}

for u in range(0, min(21, CONFIG['u_max'] + 1)):
    for v in range(u, min(CONFIG['n'] * u + 1, CONFIG['v_max'] + 1)):
        if multiplicity_matrix[u, v] > 0:
            key = f"{'even' if u % 2 == 0 else 'odd'}_{'even' if v % 2 == 0 else 'odd'}"
            parity_counts[key] += multiplicity_matrix[u, v]

print(f"Total solutions by (u parity, v parity):")
for key, count in parity_counts.items():
    print(f"  {key}: {count}")

In [ ]:
# Growth rate analysis
print("\nGrowth Rate Analysis:")
print("=" * 50)

u_test = [10, 20, 30, 40, 50]
max_multiplicities = []

for u in u_test:
    if u <= CONFIG['u_max']:
        v_range_test = range(u, min(CONFIG['n'] * u + 1, CONFIG['v_max'] + 1))
        max_m = max([multiplicity_matrix[u, v] for v in v_range_test], default=0)
        max_multiplicities.append(max_m)
        print(f"u={u:2d}: max M(u,v) = {max_m}")

# Fit polynomial growth
if len(max_multiplicities) > 2:
    coeffs = np.polyfit(u_test[:len(max_multiplicities)], max_multiplicities, 2)
    print(f"\nApproximate growth: M ~ {coeffs[0]:.4f}*u^2 + {coeffs[1]:.4f}*u + {coeffs[2]:.4f}")

In [ ]:
# Plot growth rate
fig, ax = plt.subplots(figsize=(10, 6))

u_plot = [u for u in u_test if u <= CONFIG['u_max']]
m_plot = max_multiplicities[:len(u_plot)]

ax.plot(u_plot, m_plot, marker='o', linewidth=2, markersize=8, label='max M(u,v)')

# Fit and plot polynomial
if len(m_plot) > 2:
    u_fit = np.linspace(min(u_plot), max(u_plot), 100)
    coeffs = np.polyfit(u_plot, m_plot, 2)
    m_fit = np.polyval(coeffs, u_fit)
    ax.plot(u_fit, m_fit, '--', alpha=0.7, label=f'Quadratic fit: {coeffs[0]:.4f}u² + {coeffs[1]:.4f}u + {coeffs[2]:.4f}')

ax.set_xlabel('u (Total Demand)', fontsize=12)
ax.set_ylabel('Maximum Multiplicity', fontsize=12)
ax.set_title('Growth Rate: Maximum M(u,v) vs u', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('figures/growth_rate.png', dpi=300, bbox_inches='tight')
plt.show()

## Notebook 6 — Essay Export Helpers

In [ ]:
# Generate summary statistics for essay
summary = {
    'n': CONFIG['n'],
    'u_max': CONFIG['u_max'],
    'v_max': CONFIG['v_max'],
    'total_computed': int(np.sum(multiplicity_matrix > 0)),
    'max_multiplicity': int(np.max(multiplicity_matrix)),
    'max_multiplicity_location': {
        'u': int(np.argmax(np.max(multiplicity_matrix, axis=1))),
        'v': int(np.argmax(np.max(multiplicity_matrix, axis=0)))
    },
    'sample_multiplicities': {}
}

# Add sample values
sample_pairs = [(10, 15), (10, 20), (20, 30), (20, 40)]
for u, v in sample_pairs:
    if u <= CONFIG['u_max'] and v <= CONFIG['v_max']:
        summary['sample_multiplicities'][f'M({u},{v})'] = int(multiplicity_matrix[u, v])

# Save summary
with open('results/summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("Summary Statistics:")
print("=" * 50)
print(json.dumps(summary, indent=2))

In [ ]:
# Generate markdown results section for essay
results_md = f"""
## Experimental Results Summary

### Configuration
- Number of variables (n): {CONFIG['n']}
- Range: u ∈ [0, {CONFIG['u_max']}], v ∈ [0, {CONFIG['v_max']}]

### Key Findings

1. **Total feasible pairs**: {summary['total_computed']} (u,v) pairs with M(u,v) > 0

2. **Maximum multiplicity**: M(u,v) = {summary['max_multiplicity']} at (u={summary['max_multiplicity_location']['u']}, v={summary['max_multiplicity_location']['v']})

3. **Sample multiplicities**:
"""

for key, value in summary['sample_multiplicities'].items():
    results_md += f"\n   - {key} = {value}"

results_md += """

### Figures Generated

1. `heatmap_log.png`: Multiplicity heatmap (log scale)
2. `heatmap_linear.png`: Multiplicity heatmap (linear scale)
3. `slices_fixed_u.png`: Slices of M(u,v) for fixed u values
4. `geometry_3d.png`: 3D scatter plots of solution vectors (n=3 only)
5. `growth_rate.png`: Growth rate analysis

### Data Exports

- `results/multiplicity_samples.csv`: Sample (u,v) pairs with multiplicities
- `results/summary.json`: Complete summary statistics
- `results/multiplicity_cache.json`: Cached computation results
"""

with open('results/results_section.md', 'w') as f:
    f.write(results_md)

print("Results section generated:")
print(results_md)

In [ ]:
print("\n" + "="*70)
print("Notebook execution complete!")
print("="*70)
print("\nGenerated files:")
print("  - figures/heatmap_log.png")
print("  - figures/heatmap_linear.png")
print("  - figures/slices_fixed_u.png")
print("  - figures/geometry_3d.png")
print("  - figures/growth_rate.png")
print("  - results/multiplicity_samples.csv")
print("  - results/summary.json")
print("  - results/results_section.md")
print("\nReady for essay integration!")